In [4]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# Load the trained fall detection model
model = load_model("falldown_pose_model.h5")

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# Class labels
class_labels = ['FALL DOWN', 'NOT FALLEN']

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the image for natural interaction
    frame = cv2.flip(frame, 1)

    # Convert to RGB for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    # Initialize prediction
    prediction_label = "No person detected"

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract landmarks
        landmarks = []
        for lm in results.pose_landmarks.landmark:
            landmarks.extend([lm.x, lm.y, lm.z])

        if len(landmarks) == 99:
            input_data = np.array(landmarks).reshape(1, 99, 1)

            # Predict using trained model
            prediction = model.predict(input_data, verbose=0)[0][0]
            class_id = 1 if prediction > 0.5 else 0
            prediction_label = f"{class_labels[class_id]} ({prediction:.2f})"

            # Draw result on frame
            color = (0, 255, 0) if class_id == 1 else (0, 0, 255)
            cv2.putText(frame, prediction_label, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

    # Display result
    cv2.imshow("Real-Time Fall Detection", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
